# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
orders = pd.read_csv('orders.csv')

orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
# Problem 1

spent_customers = orders.groupby('CustomerID')['amount_spent'].agg('sum').reset_index()

spent_customers.head()

,CustomerID,amount_spent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40


In [4]:
# Problem 2

vip = spent_customers[spent_customers['amount_spent'] > spent_customers['amount_spent'].quantile(0.95)]

preferred = spent_customers[(spent_customers['amount_spent'] >= spent_customers['amount_spent'].quantile(0.75)) & (spent_customers['amount_spent'] <= spent_customers['amount_spent'].quantile(0.95))]

display(preferred.sort_values(by = ['amount_spent'],ascending = False).head())

display(vip.sort_values(by = ['amount_spent'],ascending = False).head())

,CustomerID,amount_spent
537,13050,5836.86
304,12720,5781.73
2105,15218,5756.89
3904,17686,5739.46
631,13178,5725.47


,CustomerID,amount_spent
1690,14646,280206.02
4202,18102,259657.30
3729,17450,194550.79
3009,16446,168472.50
1880,14911,143825.06


In [5]:
# Problem 3

spent_customers['type'] = pd.qcut(spent_customers['amount_spent'], [0, 0.75, 0.95, 1], labels = ['General', 'Preferred', 'VIP'])

spent_customers.head()

,CustomerID,amount_spent,type
0,12346,77183.60,VIP
1,12347,4310.00,Preferred
2,12348,1797.24,Preferred
3,12349,1757.55,Preferred
4,12350,334.40,General


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [23]:
Q2 = pd.merge(left = spent_customers, right = orders[['CustomerID','Country']], how = 'left', left_on = 'CustomerID', right_on = 'CustomerID').drop_duplicates('CustomerID').drop(columns = 'amount_spent')

Q2 = Q2[Q2['type'] == 'VIP'].drop(columns = 'CustomerID')

Q2.value_counts()

type  Country        
VIP   United Kingdom     177
      Germany             10
      France               9
      Switzerland          3
      Spain                2
      Portugal             2
      Australia            2
      EIRE                 2
      Japan                2
      Finland              1
      Channel Islands      1
      Netherlands          1
      Norway               1
      Singapore            1
      Denmark              1
      Sweden               1
      Cyprus               1
dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [49]:
Q3 = pd.merge(left = spent_customers, right = orders[['CustomerID','Country']], how = 'left', left_on = 'CustomerID', right_on = 'CustomerID').drop_duplicates('CustomerID').drop(columns = 'amount_spent')

Q3 = Q3[Q3['type'] == 'Preferred'].drop(columns = 'CustomerID')

Q3 =pd.concat([Q2,Q3]).drop(columns = 'type')

Q3.value_counts()

Country        
United Kingdom     932
Germany             39
France              29
Belgium             11
Switzerland          9
Spain                7
Portugal             7
Norway               7
Italy                5
Finland              5
Japan                4
Australia            4
Channel Islands      4
EIRE                 3
Denmark              3
Cyprus               3
Sweden               2
Israel               2
Iceland              1
Austria              1
Greece               1
Lebanon              1
Malta                1
Netherlands          1
Poland               1
Singapore            1
Canada               1
dtype: int64